In [3]:
from img_xtend.utils.triton import TritonRemoteModel 


url = "http://localhost:8000/osnet_x1_0"

model = TritonRemoteModel(url)


In [4]:
import onnx

local_model = onnx.load("/home/nvidia/dev/img_new/models_weights/reidentification/osnet_x1_0.onnx")


In [5]:
import torch
device = 'cpu'

dummy_input = torch.randn(1, 3, 256, 128).to(device).half()


In [33]:
import onnxruntime
import numpy as np

device = 'cpu'

onnx_path = '/home/nvidia/dev/img_new/models_weights/reidentification/osnet_x1_0.onnx'

ort_session = onnxruntime.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

dummy_input = np.random.randn(1, 3, 256, 128).astype(np.float16)
dummy_input = np.ones((1, 3, 256, 128)).astype(np.float16)
# dummy_input = torch.randn(1, 3, 256, 128).to(device).half()


def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input}
ort_outs = ort_session.run(None, ort_inputs)

output_triton = model(dummy_input)[0]
print(output_triton.shape)
print(output_triton[0,:10])
print('output onnx')
print(type(ort_outs[0]))
print(ort_outs[0].shape)
print(ort_outs[0][0,:10])

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(output_triton, ort_outs[0],rtol=1e-1, atol=1e-01)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")


(1, 512)
[0.     1.027  0.     0.     2.092  0.0921 1.668  0.     0.2192 0.662 ]
output onnx
<class 'numpy.ndarray'>
(1, 512)
[ 0.      0.      0.      0.      0.      0.      0.      0.1388  1.845
 11.35  ]


AssertionError: 
Not equal to tolerance rtol=0.1, atol=0.1

Mismatched elements: 376 / 512 (73.4%)
Max absolute difference: 26.3
Max relative difference: 8.52
 x: array([[0.0000e+00, 1.0273e+00, 0.0000e+00, 0.0000e+00, 2.0918e+00,
        9.2102e-02, 1.6680e+00, 0.0000e+00, 2.1924e-01, 6.6211e-01,
        2.5513e-01, 1.2031e+00, 0.0000e+00, 3.1006e-01, 0.0000e+00,...
 y: array([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.3879e-01, 1.8447e+00, 1.1352e+01,
        0.0000e+00, 0.0000e+00, 1.3055e+01, 3.4609e+00, 2.0625e+00,...

In [30]:
path = "/home/nvidia/dev/img_new/models_weights/reidentification/osnet_x1_0_msmt17.pt"

model_torch = torch.load(path)

print(len(model_torch.keys()))

567
